# Read, Write and Validate HW

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ReadWriteValHW').getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print('You are working with', cores, 'core(s)')
spark

You are working with 1 core(s)


In [2]:
path = 'DataSets/'
pga = spark.read.csv( path + 'pga_tour_historical.csv', inferSchema = True, header = True)

In [3]:
pga.show(5)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows



In [4]:
pga.limit(5).toPandas()

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


In [5]:
print(pga.printSchema())
print("")
print(pga.columns)
print("")
print(pga.describe())

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)

None

['Player Name', 'Season', 'Statistic', 'Variable', 'Value']

DataFrame[summary: string, Player Name: string, Season: string, Statistic: string, Variable: string, Value: string]


In [6]:
# Edit the schema during the read
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [7]:
data_schema = [StructField('Player Name', StringType(), True),
               StructField('Season', IntegerType(), True),
               StructField('Statistic', StringType(), True),
               StructField('Variable', StringType(), True),
               StructField('Value', IntegerType(), True),
              ]

In [8]:
final_struc = StructType(fields = data_schema)

In [9]:
pga = spark.read.csv(path + 'pga_tour_historical.csv', schema = final_struc)

In [10]:
pga.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)



In [11]:
pga.describe(['value']).show()

+-------+------------------+
|summary|             value|
+-------+------------------+
|  count|           1657247|
|   mean|12494.388998743096|
| stddev| 157274.7567357075|
|    min|              -178|
|    max|           3564954|
+-------+------------------+



In [12]:
pga.select('Season', 'value').summary('count', 'min', 'max').show()

+-------+-------+-------+
|summary| Season|  value|
+-------+-------+-------+
|  count|2740403|1657247|
|    min|   2010|   -178|
|    max|   2018|3564954|
+-------+-------+-------+



In [13]:
df = pga.select('Season', 'Value')
df.write.mode('overwrite').parquet('partition_parquet/')

Py4JJavaError: An error occurred while calling o79.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)

In [14]:
df.write.mode('overwrite').partitionBy('Season').parquet('partitioned_parquet/')
df.show(5)

Py4JJavaError: An error occurred while calling o86.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)

In [15]:
path = 'partitioned_parquet/'
partitioned = spark.read.parquet(path + 'Season=2010/',
                                path + 'Season=2011/',
                                path + 'Season=2012/')
partitioned.show(5)

AnalysisException: Path does not exist: file:/C:/Users/jerf/curso udemy/PySpark/partitioned_parquet/Season=2010

In [ ]:
path = 'partitioned_parquet/'
partitioned = spark.read.option('basePath', path).parquet(path + 'Season=2010/',
                                path + 'Season=2011/',
                                path + 'Season=2012/')
partitioned.show(5)

In [16]:
values = [('Kyle', 10, 'A', 1),
         ('Melbourne', 36, 'A', 1),
         ('Nina', 123, 'A', 1),
         ('Stephen', 48, 'B', 2),
         ('Orphan', 16, 'B', 2),
         ('Imran', 1, 'B', 2)]
df = spark.createDataFrame(values, ['name', 'age', 'AB', 'Number'])
df.show()

+---------+---+---+------+
|     name|age| AB|Number|
+---------+---+---+------+
|     Kyle| 10|  A|     1|
|Melbourne| 36|  A|     1|
|     Nina|123|  A|     1|
|  Stephen| 48|  B|     2|
|   Orphan| 16|  B|     2|
|    Imran|  1|  B|     2|
+---------+---+---+------+



In [17]:
df.toPandas()

,name,age,AB,Number
0,Kyle,10,A,1
1,Melbourne,36,A,1
2,Nina,123,A,1
3,Stephen,48,B,2
4,Orphan,16,B,2
5,Imran,1,B,2
